In [ ]:
!pip install python-telegram-bot scikit-surprise pandas nest_asyncio
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import urllib.request
import zipfile
import os
import ssl
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
import nest_asyncio


nest_asyncio.apply()


# Create an unverified SSL context
ssl._create_default_https_context = ssl._create_unverified_context


url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
urllib.request.urlretrieve(url, 'ml-latest-small.zip')


with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall()


movies_file = os.path.join('ml-latest-small', 'movies.csv')
ratings_file = os.path.join('ml-latest-small', 'ratings.csv')


movies = pd.read_csv(movies_file)
ratings = pd.read_csv(ratings_file)


reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


trainset, testset = train_test_split(data, test_size=0.2)


algo = SVD()
algo.fit(trainset)


def get_movie_recommendations(algo, movie_title, movies, ratings, num_recommendations=5):
    try:
        movie_id = movies[movies['title'] == movie_title]['movieId'].values[0]
    except IndexError:
        return []


    users_who_rated_movie = ratings[ratings['movieId'] == movie_id]['userId'].unique()
    other_movie_ids = ratings[ratings['userId'].isin(users_who_rated_movie)]['movieId'].unique()
    predicted_ratings = []
    for movie in other_movie_ids:
        if movie != movie_id:
            predicted_ratings.append((movie, algo.predict(uid=0, iid=movie).est))
    predicted_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n_movies = [movie_id for movie_id, rating in predicted_ratings[:num_recommendations]]
    recommended_movie_titles = movies[movies['movieId'].isin(top_n_movies)]['title'].tolist()
    return recommended_movie_titles


# Token for your bot (replace with your token)
TOKEN = '7218884331:AAHFxiiP8fwgiYZOj5ehE1u_qh6x-z-GeQc'


# Function to start the bot
async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text('Hello! Send me the name of a movie and I will recommend other movies for you.')


# Function to handle text messages
async def handle_message(update: Update, context: CallbackContext) -> None:
    movie_title = update.message.text
    recommendations = get_movie_recommendations(algo, movie_title, movies, ratings)
    if recommendations:
        response = f"Recommendations for '{movie_title}':\n" + "\n".join(recommendations)
    else:
        response = f"Sorry, I couldn't find any recommendations for '{movie_title}'."
    await update.message.reply_text(response)


# Bot configuration
def main():
    application = Application.builder().token(TOKEN).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    application.run_polling()


if __name__ == '__main__':
    main()

ERROR:telegram.ext.Updater:Error while getting Updates: httpx.ReadError: 
ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/httpx/_transports/default.py", line 69, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.10/dist-packages/httpx/_transports/default.py", line 373, in handle_async_request
    resp = await self._pool.handle_async_request(req)
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_async/connection_pool.py", line 216, in handle_async_request
    raise exc from None
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_async/connection_pool.py", line 196, in handle_async_request
    response = await connection.handle_async_request(
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_async/connection.py", line 101, in handle_async_request
    return await self._connection.handle_async_request(request)
  File "/usr/local/li